In [1]:
import numpy as np
import pandas as pd
import cv2

import re
from pathlib import Path

from myutils.saveload import SaveLoad, NotLoaded, ModeEnum
from yesc.utils import mp_hands, open_video

In [2]:
from dotenv import dotenv_values

config = dotenv_values(".config")
data_path = Path(config["DATA_PATH"])
video_path = data_path/'raw/v1'
dataset_path = data_path/'processed/v1'

In [3]:
MODE = ModeEnum.PAUSE
SAVE = True

dataset_builder = SaveLoad(str(dataset_path/'Landmark Dataset.csv'), MODE)

In [4]:
@dataset_builder.load
def load_dataset(target):
    try:
        return pd.read_csv(target)
    except FileNotFoundError:
        raise NotLoaded

In [5]:
@dataset_builder.create
def create_dataset():
    hand_landmark_names = ("WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP", "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP",
                           "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP", )

    dataset = pd.DataFrame([], columns=('HANDEDNESS', 'SCORE',
                                        *((n+pfx) for n in hand_landmark_names for pfx in ('_X', '_Y', '_Z')), 'IS_Y'))

    return dataset


In [6]:
SELFIE_INPUT = True


@dataset_builder.process
def process_videos(landmark_ds):
    targets = [re.findall('((y|other)_(left|right)_\w+.mp4)', file.name)
             for file in video_path.iterdir()]

    with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7) as hands:
        for target in targets:
            if not len(target):
                continue
            target = target[0]
            
            filename = target[0]
            is_left = target[2] == 'left'
            is_positive = target[1] == 'y'

            print(f'Processing {filename}:', end=' ')
            landmarks = []
            with open_video(str(video_path/filename)) as video:
                for frame in video:
                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    if (SELFIE_INPUT):
                        image = cv2.flip(image, 1)
                    results = hands.process(image)
                    if not results.multi_hand_landmarks:
                        continue
                    for _handedness, hand_landmarks in zip(results.multi_handedness, results.multi_hand_landmarks):
                        handedness = _handedness.classification[0]
                        landmarks.append((handedness, hand_landmarks.landmark))

            lm_results = [(h.label, h.score, *((lm.x, lm.y, lm.z)
                        for lm in lms)) for h, lms in landmarks]
            lm_results = [(lms[0], lms[1], *np.ravel(lms[2:]), is_positive) for lms in lm_results]

            new_ds = pd.DataFrame(lm_results, columns=landmark_ds.columns)
            new_ds['IS_Y'] = is_positive
            false_indexes = new_ds[new_ds.HANDEDNESS !=
                                    ('Left' if is_left else 'Right')].index
            new_ds = new_ds.drop(false_indexes)
            initial_len = len(landmark_ds.index)
            landmark_ds = pd.concat((landmark_ds, new_ds))
            landmark_ds = landmark_ds.drop_duplicates()
            print(f"Added {len(landmark_ds.index) - initial_len} rows")

    return landmark_ds

In [7]:
@dataset_builder.save
def save_dataset(target, dataset):
    dataset.to_csv(target, index=None)

In [8]:
landmark_ds = dataset_builder.execute()

Processing other_left_jordi.mp4: Added 552 rows
Processing other_right_jordi.mp4: Added 1909 rows
Processing other_right_yudis.mp4: Added 123 rows
Processing y_left_dika.mp4: Added 572 rows
Processing y_left_jordi.mp4: Added 190 rows
Processing y_left_yudis.mp4: Added 66 rows
Processing y_right_dika.mp4: Added 489 rows
Processing y_right_jordi.mp4: Added 337 rows
Processing y_right_yudis.mp4: Added 237 rows


In [9]:
landmark_ds

,HANDEDNESS,SCORE,WRIST_X,WRIST_Y,WRIST_Z,THUMB_CMC_X,THUMB_CMC_Y,THUMB_CMC_Z,THUMB_MCP_X,THUMB_MCP_Y,...,PINKY_PIP_X,PINKY_PIP_Y,PINKY_PIP_Z,PINKY_DIP_X,PINKY_DIP_Y,PINKY_DIP_Z,PINKY_TIP_X,PINKY_TIP_Y,PINKY_TIP_Z,IS_Y
0,Left,0.998447,0.426867,0.726470,-3.355244e-07,0.475085,0.737292,-0.023625,0.528597,0.722908,...,0.418943,0.521020,-0.062958,0.412330,0.489451,-0.069021,0.409425,0.455898,-0.068255,True
1,Left,0.996158,0.420599,0.726238,-4.918656e-07,0.480033,0.741861,-0.022575,0.530826,0.722220,...,0.407161,0.489169,-0.051421,0.394217,0.445800,-0.055259,0.384253,0.403706,-0.054308,True
2,Left,0.993747,0.415386,0.717941,-5.488089e-07,0.476937,0.729452,-0.019920,0.530574,0.707565,...,0.401667,0.481976,-0.053663,0.387381,0.442011,-0.057296,0.374988,0.402892,-0.056446,True
3,Left,0.998626,0.425512,0.706939,-4.548655e-07,0.471253,0.712519,-0.023218,0.519977,0.688662,...,0.401670,0.494613,-0.055310,0.389583,0.461724,-0.059701,0.380573,0.425952,-0.058264,True
4,Left,0.993242,0.412173,0.703998,-5.492765e-07,0.473183,0.716121,-0.019831,0.526339,0.695925,...,0.398462,0.470240,-0.052493,0.383506,0.429724,-0.056155,0.371041,0.387607,-0.055224,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6816,Right,0.998665,0.682950,0.863219,-6.253512e-07,0.617191,0.843722,-0.011635,0.562743,0.784395,...,0.667021,0.612406,-0.030781,0.664994,0.678968,-0.019838,0.672989,0.699233,-0.006584,False
6817,Right,0.998455,0.683229,0.863022,-6.309974e-07,0.618180,0.844347,-0.010643,0.562980,0.784930,...,0.666417,0.612385,-0.032350,0.664423,0.678974,-0.021458,0.672791,0.699167,-0.008306,False
6818,Right,0.998610,0.683107,0.864352,-6.302539e-07,0.618879,0.846128,-0.011700,0.564296,0.787640,...,0.667158,0.614894,-0.032611,0.664829,0.680746,-0.021465,0.672903,0.700075,-0.008132,False
6819,Right,0.998744,0.683921,0.861343,-6.297710e-07,0.618447,0.842592,-0.011938,0.564191,0.785130,...,0.667187,0.614505,-0.033101,0.665571,0.679846,-0.022016,0.673722,0.697112,-0.008661,False
